In [ ]:
# summarizes statistics by subdomain (avg bias, rmse, etc)
# from the model results output by the 'analyze.py' procedure in pyap
# GO 20230613
# data in: CTDcast_metrics_hindcast.pickle (output from pyap analyze.py - not scores.py)
#       - note the above must be generated w/ mods made to pyap by GO
# data out: csv of summary stats (means by subdomain and depth class)

# 20230801 - only the means, biases, rmse's can be calculated by averaging the pre-calc stats from all ctds


In [2]:
import scipy.io
import os
import pickle as pickle
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import sys
import math
import pickle
import warnings

In [25]:
# code to check length (num of records)

# data output as pickle files by pyap
# path1 = 'D:/temp_nemo/RUN203_PLOTS_SCORES/'
#path1 = 'D:/temp_nemo/RUN203_PLOTS_SCORES/'
#path1 = 'D:/temp_nemo/RUN100/'
path1 = 'D:/nemo_outputs/ANALYSIS/SalishSea1500-RUN216/PROCESS/CTD/'
#path1 = 'D:/nemo_outputs/ANALYSIS/SalishSea1500-RUN203/PROCESS/CTD/'


# pic_data = "CTDcast_metrics_hindcast_manydepths.pickle"
pic_data = "CTDcast_metrics_hindcast_fewdepths.pickle"

scores = pickle.load(open(os.path.join(path1,pic_data), 'rb'))
#modelrun = 'SalishSea1500-RUN203'
#modelrun = 'SalishSea1500-RUN100'
modelrun = 'SalishSea1500-RUN216'
#modelrun = 'SalishSea500-201905'

limitn = 1000000 # for debug
scores_data = scores[modelrun]['scores']
sdomains = ['DI','SGN','SGS','GI','HS','JFS','PS']
stats=['bias', 'rmse', 'crmse', 'skill1981', 'mean_obs', 'mean_mod', 'std_obs', 'std_mod']
# dep_levs = ['Full', '0->2', '2->10', '10->30', '30->50', '50->100', '100->200', '200->400', '>400']
# reduced for pub 20230803
dep_levs = ['Full', '0->30', '30->150', '>150']
vars_ts = ['T','S']

In [10]:
print(scores[modelrun]['class4'].keys())

dict_keys(['Nanoose_0_CastCTD_1991-01-03_1804h', 'Nanoose_0_CastCTD_1991-01-09_1725h', 'Nanoose_0_CastCTD_1991-01-10_1715h', 'Nanoose_0_CastCTD_1991-01-12_1735h', 'Nanoose_0_CastCTD_1991-01-14_1727h', 'Nanoose_0_CastCTD_1991-01-15_1924h', 'Nanoose_0_CastCTD_1991-01-17_1836h', 'Nanoose_0_CastCTD_1991-01-18_1904h', 'Nanoose_0_CastCTD_1991-01-23_1638h', 'Nanoose_0_CastCTD_1991-01-24_1654h', 'Nanoose_0_CastCTD_1991-01-25_1902h', 'Nanoose_0_CastCTD_1991-01-29_2137h', 'Nanoose_0_CastCTD_1991-01-30_1506h', 'Nanoose_0_CastCTD_1991-02-05_1650h', 'Nanoose_0_CastCTD_1991-02-08_1710h', 'Nanoose_0_CastCTD_1991-02-12_1610h', 'Nanoose_0_CastCTD_1991-02-13_1757h', 'Nanoose_0_CastCTD_1991-02-15_1647h', 'Nanoose_0_CastCTD_1991-02-21_1646h', 'Nanoose_0_CastCTD_1991-02-22_1751h', 'Nanoose_0_CastCTD_1991-02-25_1750h', 'Nanoose_0_CastCTD_1991-02-28_1034h', 'Nanoose_0_CastCTD_1991-03-01_1632h', 'Nanoose_0_CastCTD_1991-03-04_2218h', 'Nanoose_0_CastCTD_1991-03-06_0215h', 'Nanoose_0_CastCTD_1991-03-07_2030h', '

In [21]:
# scores[modelrun]['scores']['Nanoose_107_CastCTD_2016-01-07_2204h'].keys()
# scores[modelrun]['scores']['Nanoose_107_CastCTD_2016-01-07_2204h']
# scores[modelrun]['scores']['Nanoose_107_CastCTD_2016-01-07_2204h']['SubDomain']
#print(scores[modelrun]['class4']['Nanoose_107_CastCTD_2016-01-07_2204h']['obs']['T'])
# print(scores[modelrun]['class4']['Nanoose_107_CastCTD_2016-01-07_2204h']['model']['T'])
# scores[modelrun]['class4']['Nanoose_107_CastCTD_2016-01-07_2204h']['dep']
#scores[modelrun]['scores']['Nanoose_107_CastCTD_2016-01-07_2204h']['bias']['0->10']
#scores[modelrun]['scores']['Nanoose_107_CastCTD_2016-01-07_2204h']['ObsTime']

[        nan  6.78570343  6.7804258   6.79082738  6.8465012   6.93260046
  6.97906337  6.94483249  6.89376009  6.86242561  6.85901389  6.87055329
  6.92692268  7.10910873  7.26549844  7.3985757   7.53792926  7.85841921
  8.13893008  8.21014234  8.44782499  8.52261613  9.05227205  9.55486271
  9.88973853  9.9094828   9.86002205  9.89544533  9.9919187  10.04736872
  9.87092753  9.83957643  9.80854396  9.77876941  9.78078276  9.76373779
  9.77391707         nan         nan]


In [22]:
 len(scores[modelrun]['class4']['Nanoose_0_CastCTD_1991-03-06_0215h']['model']['T'])

39

In [7]:
# for each variable (T,S)
# for each stat
# for each domain 
# for each depth level
# for each cast
# calculate the mean_obs - mean_obs_all

# using the stats (eg mean of all) 
# for each CTD, get the score (stat) for the dep strat if ctd in domain
# get the model and get the obs values (pre-calc'd avg within depth strat), 
# add to list / array 
# then calc the standard dev, crmse, WSS, R
def get_modobsvals_CTD(scores_data, var, sdomain, dep_lev):
    mod = [] # hold the vals for dep strat (mod_mean_one)
    obs = [] 
    val_n = 0 # not needed?
    n = 0
    bad = 0
    
    for ctd in scores[modelrun]['scores'].keys(): 
        if not scores[modelrun]['scores'][ctd]:
            n+=1
            
            continue
        if not scores[modelrun]['scores'][ctd]['mean_mod'][dep_lev][var]:
            bad+=1
            continue
        else:
            if scores[modelrun]['scores'][ctd]['SubDomain']==sdomain:
                
                # treating the means across deps (pre done) as single observation
                
                mean_mod_one = scores[modelrun]['scores'][ctd]['mean_mod'][dep_lev][var]
                mean_obs_one = scores[modelrun]['scores'][ctd]['mean_obs'][dep_lev][var]
                
                if not math.isnan(mean_mod_one) and not math.isnan(mean_obs_one):
                    mod.append(mean_mod_one)
                    obs.append(mean_obs_one)
                    val_n += 1
        n+=1
        # debug switch
#         if n > limitn:
#             break

    return mod, obs, val_n
    
# modified from pkg_statistics.py
def rmse(obs, mod, axis=0):
    """ Calculate RMSE = sqrt (1/n sum (mod-obs)^2 ) for an obs-mod pair. """
    return np.nanmean((np.asarray(mod) - np.asarray(obs)) ** 2, axis=axis) ** 0.5


def bias(obs, mod, axis=0):
    return np.nanmean(np.asarray(mod), axis=axis) - np.nanmean(np.asarray(obs), axis=axis)

###################################################
# crosschecking of crmse - works out - GO 20230801
def crmse(obs, mod, axis=0):
    return np.nanstd(np.asarray(mod) - np.asarray(obs), axis=axis)

# from karna p. 5736 eqn 2 (contains error, see below!)
def crmse2(obs, mod, axis=0):
    mod = np.asarray(mod)
    obs = np.asarray(obs)
    R = sp.stats.pearsonr(mod, obs)[0]
    
    # error in karna!
    #crmse = (np.nanstd(mod) ** 2 + np.nanstd(obs) ** 2 - np.nanstd(mod) * np.nanstd(obs) * R) ** 0.5
    # vs https://pcmdi.llnl.gov/staff/taylor/CV/Taylor_diagram_primer.pdf -->
    crmse = (np.nanstd(mod) ** 2 + np.nanstd(obs) ** 2 - 2 * np.nanstd(mod) * np.nanstd(obs) * R) ** 0.5
    return crmse

def crmse3(obs, mod, axis=0):
    mod = np.asarray(mod)
    obs = np.asarray(obs)
    crmse = (np.sum(((mod - np.nanmean(mod)) - (obs - np.nanmean(obs)))**2)/len(obs))**0.5
    return crmse


###################################################
# crosschecking of pearsons r cor coef - works out - GO 20230801
def pearson (obs, mod):
    # nan filter not working, nans shouldn't be here anyway -GO
#     i = ~np.isnan(np.asarray(mod)-np.asarray(obs))
#     print(i)
#     print(np.asarray(mod[i]))
    return sp.stats.pearsonr(np.asarray(mod), np.asarray(obs))   # no nanpearson

def pearson2 (obs, mod):
    mod = np.asarray(mod)
    obs = np.asarray(obs)
    R = (1 / (np.nanstd(mod) * np.nanstd(obs)) ) * (np.sum( (mod - np.nanmean(mod)) * (obs - np.nanmean(obs)) )  / len(obs))
    return R



def mae(obs, mod, axis=0):
    return np.nanmean(np.abs(np.asarray(mod) - np.asarray(obs)), axis=axis)

def Willmott1981(obs, mod, axis=0):
    
    mod = np.asarray(mod) # add by GO
    obs = np.asarray(obs)
    
    num = np.nansum((mod - obs) ** 2, axis=axis)
    obs_mean = np.nanmean(obs, axis=axis)
    dM = np.abs(mod - obs_mean)
    dO = np.abs(obs - obs_mean)
    den = np.nansum((dM + dO) ** 2, axis=axis)
    if den == 0:
        return np.nan
    else:
        return np.max([0, 1 - num / den])
    
def ncrmse1(obs, mod, axis=0):
    return (np.nanstd(np.asarray(mod) - np.asarray(obs), axis=axis)) / np.nanstd(np.asarray(obs), axis=axis)   

In [26]:
cnt_n_col = ["n obs"]
i = 0

# loop over the class4 data and calculate stats
row_labels = ["sd_levs"]
for sdomain in sdomains:
    for dep_lev in dep_levs:
        row_labels.append(sdomain + "_" + dep_lev)
all_metrics_cols = row_labels

for var in vars_ts:
    
    print(var)
    # empty arrays w/ col headings
    sdom_all = ['sdom']
    n_obs_all = ['n_obs']
    mean_m_all2 = [var + '_mean_mod'] 
    mean_o_all2 = [var + '_mean_obs']
    bias_all2 = [var + '_bias']
    std_m_all = [var + '_std_mod'] 
    std_o_all = [var + '_std_obs']
    rmse_all = [var + '_rmse']
    crmse_all = [var + '_crmse']
    nrmse_all = [var + '_nrmse']
    ncrmse_all = [var + '_ncrmse']
    pears_all = [var + '_pears']
    wss_all = [var + '_wss']
    
    for sdomain in sdomains:
        print(sdomain)
        for dep_lev in dep_levs:
            # get set of vals from all casts
            # (treating the means within dep_lev as single val)
            mod, obs, val_n = get_modobsvals_CTD(scores_data, var, sdomain, dep_lev)
            sdom_all.append(sdomain)
            n_obs_all.append(val_n)
           
            #err, mean_m_all, mean_o_all = get_means_all(var,sdomain,dep_lev,means_df)
            # I expect to see RuntimeWarnings in this block
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                mean_m = np.nanmean(np.asarray(mod))
                mean_o = np.nanmean(np.asarray(obs))
            
            if np.isnan(mean_o) or np.isnan(mean_m) or val_n == 0:
                mean_m_all2.append(np.nan); mean_o_all2.append(np.nan)
                bias_all2.append(np.nan)
                std_m_all.append(np.nan); std_o_all.append(np.nan)
                rmse_all.append(np.nan); crmse_all.append(np.nan)
                nrmse_all.append(np.nan); ncrmse_all.append(np.nan)
                pears_all.append(np.nan); wss_all.append(np.nan)
            else:
                mean_m = np.nanmean(np.asarray(mod)); mean_m_all2.append(mean_m)
                mean_o = np.nanmean(np.asarray(obs)); mean_o_all2.append(mean_o)
                bias_ = bias(obs,mod); bias_all2.append(bias_)
                
                # calculate std_dev mod, std_dev obs
                std_m = np.nanstd(np.asarray(mod)); std_m_all.append(std_m)
                std_o = np.nanstd(np.asarray(obs)); std_o_all.append(std_o)            

                # xcheck mean, bias - checks out but mod slightly differnet b/c blind
                #print(np.nanmean(np.asarray(obs)))
                rmse_ = rmse(obs, mod); rmse_all.append(rmse_)
                crmse_ = crmse(obs, mod); crmse_all.append(crmse_)
                nrmse_ =  rmse_ / std_o; nrmse_all.append(nrmse_)
                ncrmse_ = crmse_ / std_o; ncrmse_all.append(ncrmse_)
                pearson_ = pearson(obs, mod)[0]; pears_all.append(pearson_)
                wss_ = Willmott1981(obs, mod); wss_all.append(wss_)

    if i == 0:
        all_metrics_cols = np.column_stack((all_metrics_cols,sdom_all,n_obs_all))
        
    all_metrics_cols = np.column_stack((all_metrics_cols,
                                        mean_m_all2, mean_o_all2, bias_all2,
                                        std_m_all, std_o_all,
                                        rmse_all, crmse_all,
                                        nrmse_all, ncrmse_all,
                                        pears_all, wss_all
                                       ))
    i+=1 # next var (T,S)
            
            
metrics_df = pd.DataFrame(data=all_metrics_cols[1:,1:], index=all_metrics_cols[1:,0], columns=all_metrics_cols[0,1:])
# metrics_df = metrics_df.apply(pd.to_numeric, errors='coerce')
for col in [x for x in metrics_df.columns if x != 'sdom']:
    metrics_df[col] = metrics_df[col].astype(float)

metrics_df = metrics_df.round(2)
metrics_df.to_csv(os.path.join(path1,modelrun + '_meanmetricssummary_fewdepths.csv'), index=True)
metrics_df

T
DI
SGN
SGS
GI
HS
JFS
PS
S
DI
SGN
SGS
GI
HS
JFS
PS


,sdom,n_obs,T_mean_mod,T_mean_obs,T_bias,T_std_mod,T_std_obs,T_rmse,T_crmse,T_nrmse,...,S_mean_obs,S_bias,S_std_mod,S_std_obs,S_rmse,S_crmse,S_nrmse,S_ncrmse,S_pears,S_wss
DI_Full,DI,3474.0,9.74,9.80,-0.06,1.32,1.12,0.63,0.63,0.56,...,29.38,-0.09,1.39,0.82,1.03,1.03,1.25,1.25,0.68,0.75
DI_0->30,DI,3465.0,10.24,10.41,-0.17,1.89,1.71,0.85,0.83,0.50,...,28.35,-0.48,0.97,0.91,1.14,1.03,1.25,1.13,0.40,0.61
DI_30->150,DI,3062.0,9.41,9.48,-0.06,0.94,0.74,0.45,0.45,0.61,...,29.72,0.25,1.04,0.47,0.85,0.81,1.82,1.74,0.66,0.67
DI_>150,DI,1118.0,9.34,9.29,0.04,0.48,0.47,0.33,0.33,0.71,...,30.61,0.26,0.54,0.23,0.46,0.38,1.98,1.62,0.81,0.68
SGN_Full,SGN,12288.0,9.40,9.48,-0.08,1.02,1.05,0.37,0.36,0.35,...,29.98,0.34,0.93,0.92,0.49,0.35,0.54,0.38,0.93,0.93
SGN_0->30,SGN,12272.0,10.13,10.51,-0.39,1.99,2.00,0.70,0.58,0.35,...,28.28,0.09,0.78,0.94,0.67,0.67,0.71,0.71,0.72,0.83
SGN_30->150,SGN,11388.0,8.95,9.09,-0.14,0.51,0.63,0.32,0.29,0.50,...,30.09,0.47,0.28,0.39,0.52,0.24,1.36,0.61,0.79,0.62
SGN_>150,SGN,8347.0,9.25,9.12,0.13,0.34,0.43,0.31,0.28,0.71,...,30.94,0.30,0.16,0.19,0.34,0.15,1.77,0.80,0.64,0.52
SGS_Full,SGS,3834.0,9.42,9.56,-0.14,1.30,1.23,0.42,0.40,0.34,...,28.84,0.38,5.06,4.54,1.45,1.40,0.32,0.31,0.96,0.98
SGS_0->30,SGS,3832.0,10.08,10.33,-0.25,1.88,1.84,0.65,0.60,0.35,...,27.40,0.44,4.83,4.34,1.60,1.54,0.37,0.35,0.95,0.97


In [29]:
metrics_df.to_clipboard(index=True, header=True)

## scraps below

In [ ]:
# filter to right row for means
dl_row = "SGN"
# get only the rows for depth cat
metrics_df.loc[metrics_df.index.str.contains(dl_row)]

In [219]:
# for each CTD, get the score (stat) for the dep strat if ctd in domain
# then calc the avg of all
# def get_mean_CTD(scores_data, var, sdomain, stat, dep_lev):
#     avg = []
#     avg_n = 0
#     n = 0
#     bad = 0
    
#     for ctd in scores[modelrun]['scores'].keys(): 
#         if not scores[modelrun]['scores'][ctd]:
#             n+=1
            
#             continue
#         if not scores[modelrun]['scores'][ctd][stat][dep_lev][var]:
#             bad+=1
#             continue
#         else:
#             if scores[modelrun]['scores'][ctd]['SubDomain']==sdomain:
#                 sval = scores[modelrun]['scores'][ctd][stat][dep_lev][var]
                
#                 if not math.isnan(sval):
#                     avg.append(sval)
#                     avg_n += 1

#         n+=1
#         if n > limitn:
#             break
    
#     #print(var, sdomain, stat, dep_lev)
#     #print("bad ", bad)
#     if avg_n == 0:
#         avg_all = np.nan
#     elif avg_n != 1:
#         avg_all = np.round(np.nanmean(avg),3)
#     else:
#         avg_all = avg
    
#     return avg_all, avg_n



In [257]:
# sdomains = ["DI"] # debug
# dep_levs = ['10->30']



# row_labels = ["sd_levs"]
# for sdomain in sdomains:
#     for dep_lev in dep_levs:
#         row_labels.append(sdomain + "_" + dep_lev)
# # print((row_labels))

# # get 'grand mean' of stats across all casts, grouped by depth strata and domain
# # - can only do this for mean, bias not std dev or others
# # (we can take the 'grand mean' but not the 'grand standard dev')

# all_means_cols = []
# cnt_n_col = ["n obs"]
# i = 0
# for var in vars_ts:
#     print(var)
#     for stat in stats:
#         if (stat != 'bias' and stat != 'mean_obs' and stat != 'mean_mod'):
#             continue
#         print(stat)
#         one_mean_col = [var + "_" + stat]
        
#         for sdomain in sdomains: 
#             avg = []
#             avg_n = 0
#             for dep_lev in dep_levs:
#                 avg, avg_n = get_mean_CTD(scores_data, var, sdomain, stat, dep_lev)
#                 one_mean_col.append(avg)
#                 if i == 0:
#                     cnt_n_col.append(avg_n)
                    
#         if len(all_means_cols) != 0:
#             all_means_cols = np.column_stack((all_means_cols, one_mean_col))
#         else: 
#             all_means_cols = row_labels
#             all_means_cols = np.column_stack((all_means_cols,cnt_n_col))
#             all_means_cols = np.column_stack((all_means_cols,one_mean_col))
#         i+=1


In [258]:
# # converting to pandas b/c its quick afaik to convert to numeric and filter quick
# # to-do: can avoid this using np?
# means_df = pd.DataFrame(data=all_means_cols[1:,1:], index=all_means_cols[1:,0], columns=all_means_cols[0,1:])
# means_df = means_df.apply(pd.to_numeric, errors='coerce')


In [202]:
# def get_means_all(var,sd,dl,means_df):
#     error = "no"

#     # filter means df by sdomain, dep_lev, var, stat
#     mean_o_col = var + "_mean_obs"
#     mean_m_col = var + "_mean_mod"

#     # get the right col for means
#     mean_o_df = means_df.loc[:, mean_o_col]
#     mean_m_df = means_df.loc[:, mean_m_col]

#     # filter to right row for means
#     dl_row = sd + '_' + dl
#     # get only the rows for depth cat
#     mean_o_df = mean_o_df.loc[mean_o_df.index.str.contains(dl_row)]
#     mean_m_df = mean_m_df.loc[mean_m_df.index.str.contains(dl_row)]
    
#     if len(mean_o_df) != 1:
#         print('why is mean of all obs for ', sdomain, ' and dep strat ', dep_lev, ' have len != 1? Skipping.')
#         error = "yes"
#         return error,0,0
#     if len(mean_m_df) != 1:
#         print('why is mean of all mod for ', sdomain, ' and dep strat ', dep_lev, ' have len != 1? Skipping.')
#         error = "yes"
#         return error,0,0
#     if np.isnan(mean_o_df.values[0]) or np.isnan(mean_m_df.values[0]):
#         error = "yes"
#         return error,0,0
    

#     mean_o_all = mean_o_df.values[0]
#     mean_m_all = mean_m_df.values[0]
    
#     return error, mean_m_all, mean_o_all

In [237]:
# for each variable (T,S)
# for each stat
# for each domain 
# for each depth level
# for each cast
# calculate the mean_obs - mean_obs_all

# using the stats (eg mean of all) 
# for each CTD, get the score (stat) for the dep strat if ctd in domain
# get the model and get the obs values (pre-calc'd avg within depth strat), 
# add to list / array 
# then calc the standard dev, crmse, WSS, R
def get_modobsvals_CTD(scores_data, var, sdomain, dep_lev):
    mod = [] # hold the vals for dep strat (mod_mean_one)
    obs = [] 
    val_n = 0 # not needed?
    n = 0
    bad = 0
    
    for ctd in scores[modelrun]['scores'].keys(): 
        if not scores[modelrun]['scores'][ctd]:
            n+=1
            
            continue
        if not scores[modelrun]['scores'][ctd][stat][dep_lev][var]:
            bad+=1
            continue
        else:
            if scores[modelrun]['scores'][ctd]['SubDomain']==sdomain:
                
                # treating the means across deps (pre done) as single observation
                
                mean_mod_one = scores[modelrun]['scores'][ctd]['mean_mod'][dep_lev][var]
                mean_obs_one = scores[modelrun]['scores'][ctd]['mean_obs'][dep_lev][var]
                
                if not math.isnan(mean_mod_one) and not math.isnan(mean_obs_one):
                    mod.append(mean_mod_one)
                    obs.append(mean_obs_one)
                    val_n += 1
        n+=1
        # debug switch
        if n > limitn:
            break

    return mod, obs, val_n
    
# modified from pkg_statistics.py
def rmse(obs, mod, axis=0):
    """ Calculate RMSE = sqrt (1/n sum (mod-obs)^2 ) for an obs-mod pair. """
    return np.nanmean((np.asarray(mod) - np.asarray(obs)) ** 2, axis=axis) ** 0.5


def bias(obs, mod, axis=0):
    return np.nanmean(np.asarray(mod), axis=axis) - np.nanmean(np.asarray(obs), axis=axis)

###################################################
# crosschecking of crmse - works out - GO 20230801
def crmse(obs, mod, axis=0):
    return np.nanstd(np.asarray(mod) - np.asarray(obs), axis=axis)

# from karna p. 5736 eqn 2 (contains error, see below!)
def crmse2(obs, mod, axis=0):
    mod = np.asarray(mod)
    obs = np.asarray(obs)
    R = sp.stats.pearsonr(mod, obs)[0]
    
    # error in karna!
    #crmse = (np.nanstd(mod) ** 2 + np.nanstd(obs) ** 2 - np.nanstd(mod) * np.nanstd(obs) * R) ** 0.5
    # vs https://pcmdi.llnl.gov/staff/taylor/CV/Taylor_diagram_primer.pdf -->
    crmse = (np.nanstd(mod) ** 2 + np.nanstd(obs) ** 2 - 2 * np.nanstd(mod) * np.nanstd(obs) * R) ** 0.5
    return crmse

def crmse3(obs, mod, axis=0):
    mod = np.asarray(mod)
    obs = np.asarray(obs)
    crmse = (np.sum(((mod - np.nanmean(mod)) - (obs - np.nanmean(obs)))**2)/len(obs))**0.5
    return crmse


###################################################
# crosschecking of pearsons r cor coef - works out - GO 20230801
def pearson (obs, mod):
    # nan filter not working, nans shouldn't be here anyway -GO
#     i = ~np.isnan(np.asarray(mod)-np.asarray(obs))
#     print(i)
#     print(np.asarray(mod[i]))
    return sp.stats.pearsonr(np.asarray(mod), np.asarray(obs))   # no nanpearson

def pearson2 (obs, mod):
    mod = np.asarray(mod)
    obs = np.asarray(obs)
    R = (1 / (np.nanstd(mod) * np.nanstd(obs)) ) * (np.sum( (mod - np.nanmean(mod)) * (obs - np.nanmean(obs)) )  / len(obs))
    return R



def mae(obs, mod, axis=0):
    return np.nanmean(np.abs(np.asarray(mod) - np.asarray(obs)), axis=axis)

def Willmott1981(obs, mod, axis=0):
    
    mod = np.asarray(mod) # add by GO
    obs = np.asarray(obs)
    
    num = np.nansum((mod - obs) ** 2, axis=axis)
    obs_mean = np.nanmean(obs, axis=axis)
    dM = np.abs(mod - obs_mean)
    dO = np.abs(obs - obs_mean)
    den = np.nansum((dM + dO) ** 2, axis=axis)
    if den == 0:
        return np.nan
    else:
        return np.max([0, 1 - num / den])
    
def ncrmse1(obs, mod, axis=0):
    return (np.nanstd(np.asarray(mod) - np.asarray(obs), axis=axis)) / np.nanstd(np.asarray(obs), axis=axis)    
    
    
    